In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style("whitegrid")

from __future__ import (unicode_literals, print_function, absolute_import,
                        division)


from ema_workbench import (Model, MultiprocessingEvaluator, SequentialEvaluator,
                           ScalarOutcome, IntegerParameter, optimize, Scenario)
from ema_workbench.em_framework.optimization import EpsilonProgress, HyperVolume
from ema_workbench.util import ema_logging
from ema_workbench import (Model, RealParameter, IntegerParameter, CategoricalParameter, ScalarOutcome, Constant)

from problem_formulation import get_model_for_problem_formulation
import matplotlib.pyplot as plt
import seaborn as sns
from ema_workbench.em_framework.samplers import sample_levers, sample_uncertainties
from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS, SequentialEvaluator

from __future__ import (unicode_literals, print_function, absolute_import,
                        division)


from ema_workbench import (Model, MultiprocessingEvaluator,
                           ScalarOutcome, IntegerParameter, optimize, Scenario)
from ema_workbench.em_framework.optimization import EpsilonProgress, HyperVolume
from ema_workbench.util import ema_logging

from problem_formulation import get_model_for_problem_formulation
import matplotlib.pyplot as plt
import seaborn as sns


if __name__ == '__main__':
    ema_logging.log_to_stderr(ema_logging.INFO)

    dike_model = get_model_for_problem_formulation(1)

    reference_values = {'Bmax': 175, 'Brate': 1.5, 'pfail': 0.5,
                        'discount rate': 3.5,
                        'ID flood wave shape': 4}
    scen1 = {}

    for key in dike_model.uncertainties:
        name_split = key.name.split('_')

        if len(name_split) == 1:
            scen1.update({key.name: reference_values[key.name]})

        else:
            scen1.update({key.name: reference_values[name_split[1]]})

    ref_scenario = Scenario('reference', **scen1)
    
    dike_model.uncertainties = [RealParameter('A.0_ID flood wave shape', 0, 133),
                   RealParameter('A1.Bmax', 30,350),
                   CategoricalParameter('A1.Brate',(0.9,1.5,1000)),
                   RealParameter('A1.pfail',0,1),
                   RealParameter('A2.Bmax', 30,350),
                   CategoricalParameter('A2.Brate',(0.9,1.5,1000)),
                   RealParameter('A2.pfail',0,1),
                   RealParameter('A3.Bmax', 30,350),
                   CategoricalParameter('A3.Brate',(0.9,1.5,1000)),
                   RealParameter('A3.pfail',0,1),
                   RealParameter('A4.Bmax', 30,350),
                   CategoricalParameter('A4.Brate',(0.9,1.5,1000)),
                   RealParameter('A4.pfail',0,1),
                   RealParameter('A5.Bmax', 30,350),
                   CategoricalParameter('A5.Brate',(0.9,1.5,1000)),
                   RealParameter('A5.pfail',0,1),
                   CategoricalParameter('discount rate',(1.5,2.5,3.5,4.5))]


    # set levers
    dike_model.levers = [IntegerParameter('A1.DikeIncrease',0,10),
                     IntegerParameter('A2.DikeIncrease',0,10),
                     IntegerParameter('A3.DikeIncrease',0,10),
                     IntegerParameter('A4.DikeIncrease',0,10),
                     IntegerParameter('A5.DikeIncrease',0,2), #capped this variable to test if the model uses these specified levers
                     CategoricalParameter('1_RfR',(0,1)),
                     CategoricalParameter('2_RfR',(0,1)),
                     CategoricalParameter('3_RfR',(0,1)),
                     CategoricalParameter('4_RfR',(0,1)),
                     CategoricalParameter('0_RfR',(0,1)),
                     IntegerParameter('EWS_DaysToThreat',0,4)]


    #specify outcomes
    # note how we need to explicitely indicate the direction
    dike_model.outcomes = [ScalarOutcome('Expected Annual Damage', kind=ScalarOutcome.MINIMIZE),
                  ScalarOutcome('Total Investment Costs', kind=ScalarOutcome.MINIMIZE),
                  ScalarOutcome('Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE)]

    convergence_metrics = [EpsilonProgress()]

    espilon = [100000,100000,0.001]

    nfe = 10

    with SequentialEvaluator(dike_model) as evaluator:
        results, convergence = evaluator.optimize(nfe=nfe,
                                                  searchover='levers',
                                                  epsilons=espilon,
                                                  convergence=convergence_metrics,
                                                  reference=ref_scenario
                                                  )

C:\Users\newbi\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


AttributeError: 'tuple' object has no attribute 'uncertainties'

In [3]:
import problem_formulation

In [4]:
model = problem_formulation.get_model_for_problem_formulation(3)

[MainProcess/INFO] model initialized


In [9]:
for l in model.levers:
    print(repr(l))

IntegerParameter('A.1_DikeIncrease', 0, 10)
IntegerParameter('A.2_DikeIncrease', 0, 10)
IntegerParameter('A.3_DikeIncrease', 0, 10)
IntegerParameter('A.4_DikeIncrease', 0, 10)
IntegerParameter('A.5_DikeIncrease', 0, 10)
IntegerParameter('0_RfR', 0, 1)
IntegerParameter('1_RfR', 0, 1)
IntegerParameter('2_RfR', 0, 1)
IntegerParameter('3_RfR', 0, 1)
IntegerParameter('4_RfR', 0, 1)
IntegerParameter('EWS_DaysToThreat', 0, 4)
